In [6]:
import glob  
name_lst=[]
for filename in glob.iglob('/media/meiyih/meiyih_datasets/COCO_TS_labels/'+"*"+'.png'):
    name=(((filename.split("/")[-1]).split(".")[0])+".jpg")
    name_lst.append(name)
    

In [8]:
import os
os.getcwd()

'/home/meiyih'

In [9]:
len(name_lst)

14690

In [31]:
jpg_name_lst=[]

In [4]:
import glob  
import os 
for filename in glob.iglob('/home/meiyih/semantic_segmentation_unet/data/val_images/'+"*"+'.jpg'):
    if 'jpg' in (((filename.split("/")[-1]).split(".")[0])+".jpg"):
        print(((filename.split("/")[-1]).split(".")[0])+".jpg") 
   
        

COCO_train2014_000000469315.jpg
COCO_train2014_000000421539.jpg
COCO_train2014_000000569433.jpg
COCO_train2014_000000391519.jpg
COCO_train2014_000000483078.jpg
COCO_train2014_000000482959.jpg
COCO_train2014_000000394447.jpg
COCO_train2014_000000442078.jpg
COCO_train2014_000000513015.jpg
COCO_train2014_000000459515.jpg
COCO_train2014_000000456500.jpg
COCO_train2014_000000395920.jpg
COCO_train2014_000000540428.jpg
COCO_train2014_000000538838.jpg
COCO_train2014_000000479102.jpg
COCO_train2014_000000503149.jpg
COCO_train2014_000000445080.jpg
COCO_train2014_000000420290.jpg
COCO_train2014_000000580906.jpg
COCO_train2014_000000418229.jpg
COCO_train2014_000000532958.jpg
COCO_train2014_000000508053.jpg
COCO_train2014_000000531488.jpg
COCO_train2014_000000468652.jpg
COCO_train2014_000000570339.jpg
COCO_train2014_000000429769.jpg
COCO_train2014_000000430932.jpg
COCO_train2014_000000420629.jpg
COCO_train2014_000000444445.jpg
COCO_train2014_000000462599.jpg
COCO_train2014_000000540728.jpg
COCO_tra

In [14]:
for filename in glob.iglob('/media/meiyih/meiyih_datasets/train2014/'+"*"+'.jpg'):
    if (((filename.split("/")[-1]).split(".")[0])+".jpg") in name_lst:
        jpg_name_lst.append(filename)
        

In [28]:
import shutil
import os

source = "/media/meiyih/meiyih_datasets/train2014"
destination = "/media/meiyih/meiyih_datasets/COCO_TS_imgs"

files = os.listdir(source)

for file in files:
    if file in name_lst:
        print(file)
        new_path = shutil.move(f"{source}/{file}", destination)
        print(new_path)

COCO_train2014_000000291898.jpg
/media/meiyih/meiyih_datasets/COCO_TS_imgs/COCO_train2014_000000291898.jpg
COCO_train2014_000000102290.jpg
/media/meiyih/meiyih_datasets/COCO_TS_imgs/COCO_train2014_000000102290.jpg
COCO_train2014_000000230996.jpg
/media/meiyih/meiyih_datasets/COCO_TS_imgs/COCO_train2014_000000230996.jpg
COCO_train2014_000000300815.jpg
/media/meiyih/meiyih_datasets/COCO_TS_imgs/COCO_train2014_000000300815.jpg
COCO_train2014_000000515728.jpg
/media/meiyih/meiyih_datasets/COCO_TS_imgs/COCO_train2014_000000515728.jpg
COCO_train2014_000000302236.jpg
/media/meiyih/meiyih_datasets/COCO_TS_imgs/COCO_train2014_000000302236.jpg
COCO_train2014_000000387517.jpg
/media/meiyih/meiyih_datasets/COCO_TS_imgs/COCO_train2014_000000387517.jpg
COCO_train2014_000000430122.jpg
/media/meiyih/meiyih_datasets/COCO_TS_imgs/COCO_train2014_000000430122.jpg
COCO_train2014_000000075740.jpg
/media/meiyih/meiyih_datasets/COCO_TS_imgs/COCO_train2014_000000075740.jpg
COCO_train2014_000000112604.jpg
/medi

In [1]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class CarvanaDataset(Dataset):#class COCODataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)#list of images in image_dir
        
        self.mask_dirs = []
        
        self.mask = os.listdir(mask_dir)#list of masks in mask_dir
        for i in range(len(self.mask)):
            mask_name = mask_dir + self.mask[i]
            self.mask_dirs.append(mask_name)

        print(self.mask_dirs)
        
        

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", ".png"))
        image = np.array(Image.open(img_path).convert("RGB"))#using numpy array here is becs we will be using augmentations lib later, which if using PIL in line 19 and 20, we need to converted img to np array
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0 #preprocess:change those mask that is 255 to 1 #(R)we are going to use sigmoid in our last activation #to ensure correctness of labels e.g. to indicate the prob of white pixels

        mask_name = self.mask_dirs[index]

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask, mask_name



In [ ]:
#Ori
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class CarvanaDataset(Dataset):#class COCODataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", ".png"))
        image = np.array(Image.open(img_path).convert("RGB"))#using numpy array here is becs we will be using augmentations lib later, which if using PIL in line 19 and 20, we need to converted img to np array
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0 #preprocess:change those mask that is 255 to 1 #(R)we are going to use sigmoid in our last activation #to ensure correctness of labels e.g. to indicate the prob of white pixels

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask



In [18]:
15%4

3

In [ ]:
import torch
import torchvision
from dataset import CarvanaDataset
from torch.utils.data import DataLoader

def save_checkpoint(state, filename="my_checkpoint2.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = CarvanaDataset( #COCODataset
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = CarvanaDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

def check_accuracy(loader, model, device="cuda"): #validation function
    num_correct = 0
    num_pixels = 0 #for segmentation we are outputting prediction for the class of each individual pixel
    dice_score = 0
    model.eval()

    with torch.no_grad():##validation function use torch.no_grad()
        for idx, (x, y, mask_name) in enumerate(loader):
#         for x, y,_ in loader:#go through the loader
            x = x.to(device)
            y = y.to(device).unsqueeze(1)#do unsqueeze(1) is becs the labels/mask is grey scale..thus, it doesnt have a channel
            preds = torch.sigmoid(model(x))#run through the model and get predictions
            preds = (preds > 0.5).float()#convert those >0.5 to 1 and less than that to 0
            num_correct += (preds == y).sum()#sum all of the pixels
            num_pixels += torch.numel(preds)#num of pixels in that img
            # dice score:element wise multiply them, if both pixels output are white (the same), then it will output 1 ; we will sum the num of pixels where they are both the same/ both outputting white pixels, then we divide by the num of pixels outputting 1 for both of them; then add 1e-8
            dice_score += (2 * (preds * y).sum()) / ( 
                (preds + y).sum() + 1e-8
            )

    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

def save_predictions_as_imgs(#visualiization
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y, mask_name) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():## save output mask of val_imgs
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/{mask_name[int(idx)%4]}_pred_{idx}.png"#the pred output eval by the model on val img #mask_name[int(idx)%4] where the (idx)%4 is becs the batch size is 4...thus,the mask_name will be a list with 4 mask's name only
        )
        #torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")#the GT/val_mask of this val img

    model.train()
    
##################################################################################################################################
##CLASSES
##################################################################################################################################
import torch
class LRScheduler():
    """
    Learning rate scheduler. If the validation loss does not decrease for the 
    given number of `patience` epochs, then the learning rate will decrease by
    by given `factor`.
    """
    def __init__(
        self, optimizer, patience=5, min_lr=1e-6, factor=0.5
    ):
        """
        new_lr = old_lr * factor
        :param optimizer: the optimizer we are using
        :param patience: how many epochs to wait before updating the lr
        :param min_lr: least lr value to reduce to while updating
        :param factor: factor by which the lr should be updated
        """
        self.optimizer = optimizer
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
        self.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
                self.optimizer,
                mode='min',
                patience=self.patience,
                factor=self.factor,
                min_lr=self.min_lr,
                verbose=True
            )
    def __call__(self, val_loss):
        self.lr_scheduler.step(val_loss)    
        
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True        